In [119]:
## Import packages
import numpy as np
import pandas as pd
from pyzillow.pyzillow import ZillowWrapper, GetDeepSearchResults, GetUpdatedPropertyDetails
import xml.etree.ElementTree as ET

In [120]:
def get_deep_comps(zw, zpid, count, rentzestimate=False):
        """
        GetDeepComps API
        zw is ZillowWrapper object
        """
        url = 'http://www.zillow.com/webservice/GetDeepComps.htm'
        params = {
                  'zws-id': zw.api_key,
                  'zpid': zpid,
                  'count': count,
                  'rentzestimate': rentzestimate
        }
        return zw.get_data(url, params)

In [121]:
## Initialize the Zillow wrapper with API key
zillow_data = ZillowWrapper('X1-ZWz198jku2j0uj_1acr8')

In [122]:
## Get details for property
address = '2684 S Lafayette St'
zipcode = '80210'
deep_search_response = zillow_data.get_deep_search_results(address,zipcode)
lafayette_2684 = GetDeepSearchResults(deep_search_response)

In [123]:
## Store zpid for property
zillow_id = int(lafayette_2684.zillow_id)

In [124]:
zillow_id

13377244

In [142]:
deep_comps_response = get_deep_comps(zillow_data,zillow_id,25)[2]

In [143]:
deep_comps_response

<Element 'response' at 0x00000000096A0958>

In [191]:
attribute_mapping = {
    'zillow_id': 'zpid',
    'latitude': 'address/latitude',
    'longitude': 'address/longitude',
    'street': 'address/street',
    'zipcode': 'address/zipcode',
    'city': 'address/city',
    'state': 'address/state',
    'tax_year': 'taxAssessmentYear',
    'tax_value': 'taxAssessment',
    'year_built': 'yearBuilt',
    'property_size': 'lotSizeSqFt',
    'home_size': 'finishedSqFt',
    'bathrooms': 'bathrooms',
    'bedrooms': 'bedrooms',
    'last_sold_date': 'lastSoldDate',
    'last_sold_price': 'lastSoldPrice',
    'zestimate_amount': 'zestimate/amount',
    'zestimate_last_updated': 'zestimate/last-updated',
    'zestimate_value_change': 'zestimate/valueChange',
    'zestimate_valuation_range_high':
    'zestimate/valuationRange/high',
    'zestimate_valuationRange_low': 'zestimate/valuationRange/low',
    'zestimate_percentile': 'zestimate/percentile',
}

def parse_property(house):
        house_dict = {}
        for key in attribute_mapping.keys():
            house_dict[key] = house.find(attribute_mapping[key]).text
        house_dict['neighborhood'] = house.find('localRealEstate/region').attrib['name']
        return house_dict
    

def parse_deep_comp_results(data):
    """
    Creates instance of GeocoderResult from the provided XML data array
    """
        
    principal = data.find('properties/principal')
    principal_dict = parse_property(principal)
    comparables = data.findall('properties/comparables/comp')
    comp_list = []
    for comp in comparables:
        comp_score = comp.attrib['score']
        comp_dict = parse_property(comp)
        comp_dict['score'] = comp_score
        comp_list.append(comp_dict)
        
    return principal_dict, comp_list
        
        

In [193]:
principal, comps = parse_deep_comp_results(deep_comps_response)

In [201]:
principal_df = pd.DataFrame(principal,index=[0])
comps_df = pd.DataFrame(comps)

In [202]:
comps_df

,bathrooms,bedrooms,city,home_size,last_sold_date,last_sold_price,latitude,longitude,neighborhood,property_size,...,tax_year,year_built,zestimate_amount,zestimate_last_updated,zestimate_percentile,zestimate_valuationRange_low,zestimate_valuation_range_high,zestimate_value_change,zillow_id,zipcode
0,2.0,3,Denver,1833,01/13/2017,430000,39.67293,-104.968458,University,6250,...,2015,1946,463904,04/20/2017,55,436070,501016,14701,13377023,80210
1,2.0,2,Denver,1582,05/25/2016,290000,39.756722,-104.934227,North Park Hill,11900,...,2015,1922,584340,04/20/2017,88,555123,619400,4103,13283732,80207
2,2.0,4,Denver,1658,08/19/2016,410000,39.67005,-104.970279,University,6250,...,2015,1942,446144,04/20/2017,52,423837,472913,5998,13377145,80210
3,1.0,2,Denver,792,09/21/2016,357000,39.675157,-104.970265,University,6250,...,2015,1923,408310,04/20/2017,43,383811,428725,8171,13376224,80210
4,2.0,3,Denver,1200,05/19/2016,320000,39.675295,-104.970264,University,6250,...,2015,1936,404332,04/20/2017,42,367942,444765,16754,13376225,80210
5,1.0,2,Denver,1167,05/04/2016,340000,39.674734,-104.973776,Rosedale,6098,...,2015,1940,378774,04/20/2017,36,352260,397713,2193,13376653,80210
6,2.0,4,Denver,2028,10/12/2016,497500,39.674704,-104.96962,University,7690,...,2015,1969,531033,04/20/2017,65,504481,562895,9597,13376863,80210
7,2.2,4,Denver,2981,07/05/2016,767000,39.673927,-104.971965,University,4791,...,2015,1946,750413,04/20/2017,85,607835,900496,-3584,13376912,80210
8,2.0,4,Denver,1600,07/05/2016,420000,39.674302,-104.973133,University,6250,...,2015,1942,451186,04/20/2017,53,428627,473745,5441,13376930,80210
9,2.0,3,Denver,1604,06/09/2016,465000,39.672407,-104.97027,University,6098,...,2015,1942,487780,04/20/2017,59,458513,512169,5548,13376994,80210
